# 🍇 GraphX - LoL Items

All the details about the challenge is in the [repository](https://github.com/avcaliani/graphx-app).

<img src="https://raw.githubusercontent.com/avcaliani/graphx-app/main/.docs/lol-graph.png" height="512px">

## PySpark

Before starting, let's configure the PySpark.

In [1]:
!pip install pyspark==3.5.0

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('graphx-app') \
    .config('spark.jars.packages', "graphframes:graphframes:0.8.3-spark3.5-s_2.12") \
    .getOrCreate()

# 👇 Required to run Pregel
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")

In [3]:
from graphframes import GraphFrame

In [4]:
items_data = [
    ("Ruby Crystal", "Basic", 400),
    ("Bami's Cinder", "Epic", 200),
    ("Cloth Armor", "Basic", 300),
    ("Chain Vest", "Epic", 500),
    ("Sunfire Aegis", "Legendary", 900),
]
# GraphFrame demands a column called "id", so in the diagram the "id" is the "name"
# https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html#graphframes.GraphFrame
items = spark.createDataFrame(data = items_data, schema = ["id", "rarity", "cost"])

items_relation_data = [
    ("Ruby Crystal", "Bami's Cinder", 2),
    ("Cloth Armor", "Chain Vest", 1),
    ("Bami's Cinder", "Sunfire Aegis", 1),
    ("Chain Vest", "Sunfire Aegis", 1),
]
items_relation = spark.createDataFrame(data = items_relation_data, schema = ["src", "dst", "amount"])

graph = GraphFrame(v=items, e=items_relation)

In [5]:
graph.vertices.show()

+-------------+---------+----+
|           id|   rarity|cost|
+-------------+---------+----+
| Ruby Crystal|    Basic| 400|
|Bami's Cinder|     Epic| 200|
|  Cloth Armor|    Basic| 300|
|   Chain Vest|     Epic| 500|
|Sunfire Aegis|Legendary| 900|
+-------------+---------+----+



In [6]:
graph.edges.show()

+-------------+-------------+------+
|          src|          dst|amount|
+-------------+-------------+------+
| Ruby Crystal|Bami's Cinder|     2|
|  Cloth Armor|   Chain Vest|     1|
|Bami's Cinder|Sunfire Aegis|     1|
|   Chain Vest|Sunfire Aegis|     1|
+-------------+-------------+------+



In [7]:
from pyspark.sql.functions import coalesce, col, lit, sum, when, array_agg
from graphframes.lib import Pregel

result = (
    graph.pregel
        .setMaxIter(3) # How deep in the graph the code will go, default is 10.
        # You must start the column, even if you will only use default values.
        # Vertex that don't receive a message will receive "null"
        .withVertexColumn(
            colName = "previous_items_cost",
            initialExpr = lit(0),
            updateAfterAggMsgsExpr = Pregel.msg()
        )
        .sendMsgToDst(
            when(
                Pregel.src("previous_items_cost").isNotNull(),
                Pregel.src("previous_items_cost") + Pregel.src("cost")
            )
            .otherwise(Pregel.src("cost")) * Pregel.edge("amount")
        )
        .aggMsgs(sum(Pregel.msg()))
        .withVertexColumn(
            colName = "total_cost",
            initialExpr = lit(0),
            updateAfterAggMsgsExpr = col("cost") + coalesce(col("previous_items_cost"), lit(0))
        )
        .run()
)

result.show(truncate = False)

+-------------+---------+----+-------------------+----------+
|id           |rarity   |cost|previous_items_cost|total_cost|
+-------------+---------+----+-------------------+----------+
|Ruby Crystal |Basic    |400 |NULL               |400       |
|Bami's Cinder|Epic     |200 |800                |1000      |
|Cloth Armor  |Basic    |300 |NULL               |300       |
|Chain Vest   |Epic     |500 |300                |800       |
|Sunfire Aegis|Legendary|900 |1800               |2700      |
+-------------+---------+----+-------------------+----------+

